# Data Mining

In this notebook, I will be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal here is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates for easy search. 

Here is a list of some of the variants in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once I have extracted these date patterns from the text, I will sort them in ascending chronological order according to the following rules:
* Assuming all dates in xx/xx/xx format are mm/dd/yy
* Assuming all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assuming it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, I will find the correct date in each note and return a pandas series with the date as index in chronological order of the date.

In [17]:
import pandas as pd
import os

date_file = os.path.join("Resources", "dates.txt")
doc = []
with open(date_file) as file:
    for line in file:
        doc.append(line)

note_ds = pd.Series(doc)
note_ds.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [18]:
#This function will extract the dates and sort them
def date_sorter():
    import re
    import numpy as np

    #Create output directory
    if not os.path.exists("Output"):
        os.mkdir("Output")
    outfile = os.path.join("Output","Dated Notes.csv")
    
    month = {'Jan':'01/',
            'Feb': '02/',
            'Mar': '03/',
            'Apr': '04/',
            'May': '05/',
            'Jun': '06/',
            'Jul': '07/',
            'Aug': '08/',
            'Sep': '09/',
            'Oct': '10/',
            'Nov': '11/',
            'Dec': '12/'}
    text = pd.DataFrame(note_ds, columns=['Medical Note'])
    text['date'] = text['Medical Note'].str.findall(r'(?:\d{1,2}[/-])?(?:\d{1,2}[/])\d{2,4}')
    text['date'] = text.apply(lambda x: np.nan if len(x['date'])==0 else x['date'][0], axis=1)
    text['modate'] = text['Medical Note'].str.findall(r'(?:\d{1,2}[/-])(?:\d{1,2}[-])\d{2,4}')
    text['modate'] = text.apply(lambda x: np.nan if len(x['modate'])==0 else x['modate'][0], axis=1)
    text['strdate'] = text['Medical Note'].str.findall(r'(?:\d{2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.,-]* (?:(?:\d{1,2})[a-z,-]* )?\d{2,4}')
    text['strdate'] = text.apply(lambda x: np.nan if len(x['strdate'])==0 else x['strdate'][0], axis=1)
    text['year'] = text['Medical Note'].str.findall(r'(?:\d{4})')
    text['year'] = text.apply(lambda x: np.nan if len(x['year'])==0 else x['year'][0], axis=1)
    text['date'].fillna(text['modate'], inplace=True)
    text['date'].fillna(text['strdate'], inplace=True)
    text['date'].fillna(text['year'], inplace=True)
    text['date'] = text['date'].str.replace(r'(\W\d{2}$)', lambda x: x.groups()[0][0]+'19'+ x.groups()[0][1:3])
    text['date'] = text['date'].str.replace(r'(^\d{4}$)', lambda x: '01/01/'+ x.groups()[0][:4])
    text['date'] = text['date'].str.replace(r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z., ]*', lambda x: month[x.groups()[0][:3]])
    text['date'] = text['date'].str.replace(r', ', '/')
    text['date'] = text['date'].str.replace(r'//', '/')
    text['date'] = text['date'].str.replace(r'(\W\d{2}) ', lambda x: x.groups()[0][:3]+'/')
    text['date'] = pd.to_datetime(text['date'])
    dates_df = text[['date','Medical Note']]
    dates_df = dates_df.sort_values('date', ascending = True)
    dates_df = dates_df.reset_index(drop = True)
    output = dates_df[["date",'Medical Note']]
    output.to_csv(outfile)
    return output

date_sorter().head()

,date,Medical Note
0,1971-04-10,(4/10/71)Score-1Audit C Score Current:\n
1,1971-05-18,5/18/71 Total time of visit (in minutes):\n
2,1971-07-08,sshe plans to move as of 7/8/71 In-Home Servic...
3,1971-07-11,7/11/71 SOS-10 Total Score:\n
4,1971-09-12,9/12/71 [report_end]\n
